<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ce220a01b38792804ef1645bc8a36d19a5af4406ff06ffbccce671c38548a6d2
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-08 11:01:48
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: 41.72 K (0.3%)
Current PnL: -17.94 L (-12.07%)
CY Booked + Current PnL: -8.08 L (-5.44%)
-------------------
Total profit:  2.89 L
Total loss:  -20.83 L
-------------------
Total Booked + Current PnL: 18.90 L (15.53%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.04%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.66 L (56.14%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.18,72.0,H-MC,7.99,143720.0,28784.0,7804.0,0.62,25.04,5.43,31.83,81.0,3.69,1.08,44.91,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-14.75,55.0,H-LC,10.44,170423.0,14246.0,11844.0,-0.74,9.12,6.95,16.71,22.0,1.20,1.28,28.15,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,104.27,61.0,M-SC,6.19,88045.0,-12732.0,12828.0,-1.11,-12.63,14.57,0.09,245.0,-0.99,0.66,15.67,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,48.0,M-LC,4.52,214358.0,12878.0,14341.0,0.28,6.39,6.69,13.51,60.0,0.90,1.61,19.79,XY25,NTT,BANKS
33,HEROMOTOCO,5949.07,4.14,82.0,H-MC,22.07,192920.0,42007.0,15299.0,2.78,27.83,7.93,37.97,93.0,2.75,1.45,60.18,AR,ATH,AUTO


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-21.57,36.0,H-MC,2.15,134854.0,-934.0,43679.0,-0.01,-0.69,32.39,31.48,107.0,-0.02,1.02,11.35,X40,ATH,INSURANCE
31,HAVELLS,2069.16,-0.09,62.0,H-MC,1.89,247759.0,28.0,75021.0,0.36,0.01,30.28,30.30,92.0,0.00,1.87,13.30,X40,ATH,ELECTRICAL
36,ICICIGI,2260.25,-20.81,40.0,H-MC,3.94,136108.0,-25.0,31155.0,-0.06,-0.02,22.89,22.86,91.0,-0.00,1.02,12.59,X40,ATH,INSURANCE
53,PGHH,17973.08,-30.41,55.0,H-MC,4.87,201300.0,480.0,68301.0,-0.55,0.24,33.93,34.25,80.0,0.01,1.52,5.45,X40,ATH,FMCG
88,WIPRO,420.00,-14.25,40.0,M-LC,6.88,150710.0,-235.0,109687.0,-0.24,-0.16,72.78,72.51,53.0,-0.00,1.13,5.86,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-2.80,31.0,H-LC,10.06,221888.0,-20036.0,83297.0,-0.40,-8.28,37.54,26.15,8.0,-0.24,1.67,4.85,X40,ATH,IT
41,INFY,2275.00,-21.39,38.0,H-LC,10.88,305783.0,-7105.0,178791.0,-0.62,-2.27,58.47,54.87,3.0,-0.04,2.30,4.23,X40,BTT,IT
76,TCS,4998.00,-28.69,38.0,H-LC,14.29,284124.0,-61832.0,185675.0,-0.84,-17.87,65.35,35.80,1.0,-0.33,2.14,0.65,X40,BTT,IT
85,VBL,671.64,-16.13,38.0,H-LC,6.90,302029.0,-13813.0,126490.0,0.80,-4.37,41.88,35.67,5.0,-0.11,2.27,8.88,X40N,ATH,FMCG
45,JIOFIN,387.00,-1.47,42.0,H-LC,13.98,217840.0,434.0,53066.0,0.58,0.20,24.36,24.61,48.0,0.01,1.64,55.13,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-6.65,48.0,H-LC,1.71,231548.0,-14603.0,93823.0,-0.96,-5.93,40.52,32.18,16.0,-0.16,1.74,27.97,X200,ATH,IT
52,NESTLEIND,1377.00,-8.73,60.0,H-LC,2.60,280964.0,15538.0,41246.0,-0.65,5.85,14.68,21.40,11.0,0.38,2.12,13.61,XY25,NTT,FMCG
43,ITC,452.00,-38.34,48.0,H-LC,3.13,199512.0,-626.0,21507.0,0.16,-0.31,10.78,10.44,4.0,-0.03,1.50,5.23,X40,NTT,FMCG
19,CIPLA,1795.00,-20.12,50.0,H-LC,5.77,213282.0,8782.0,32632.0,0.22,4.29,15.30,20.25,10.0,0.27,1.61,13.40,X40N,BTT,PHARMA
85,VBL,671.64,-16.13,38.0,H-LC,6.90,302029.0,-13813.0,126490.0,0.80,-4.37,41.88,35.67,5.0,-0.11,2.27,8.88,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SHALBY,327.0,1097.77,79.0,M-SC,3.24,162862.0,-18281.0,63744.0,7.09,-10.09,39.14,25.10,235.0,-0.29,1.23,30.52,XY24,NTT,HEALTHCARE
84,VALIANTORG,838.0,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS
55,RAJESHEXPO,518.0,1737.12,57.0,L-SC,8.33,50138.0,-87039.0,87130.0,4.65,-63.45,173.78,0.07,267.0,-1.00,0.38,24.70,OX40N,NTT,JEWELLERY
69,SONACOMS,1006.0,-35.77,53.0,M-SC,12.46,80019.0,-21140.0,96031.0,3.66,-20.90,120.01,74.03,202.0,-0.22,0.60,12.83,AR,BTT,AUTO
61,SAIL,228.0,45.15,65.0,M-MC,7.54,228923.0,3961.0,163703.0,2.85,1.76,71.51,74.53,192.0,0.02,1.72,32.91,XY24,BTT,STEEL


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.31,188600.0,23930.0,44038.0,-2.96,14.53,23.35,41.28,89.0,0.54,1.42,23.67,X40N,ATH,AC
22,COLPAL,3767.14,3.80,61.0,H-MC,8.70,189017.0,-22724.0,104810.0,-1.68,-10.73,55.45,38.77,84.0,-0.22,1.42,12.50,XY25,ATH,FMCG
39,INDIGOPNTS,1408.00,113.17,48.0,M-SC,3.56,139946.0,-34613.0,34651.0,-1.47,-19.83,24.76,0.02,221.0,-1.00,1.05,21.73,OX40N,NTT,PAINTS
48,LAOPALA,464.00,110.83,45.0,H-SC,3.84,66989.0,-33857.0,59681.0,-1.41,-33.57,89.09,25.61,142.0,-0.57,0.50,24.20,AR,NTT,CERAMICS
7,ATULAUTO,844.00,3927.03,76.0,M-SC,8.64,125576.0,-21792.0,73613.0,-1.23,-14.79,58.62,35.16,236.0,-0.30,0.95,28.26,XY24,NTT,AUTO


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,104.27,61.0,M-SC,6.19,88045.0,-12732.0,12828.0,-1.11,-12.63,14.57,0.09,245.0,-0.99,0.66,15.67,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,85.53,48.0,H-SC,12.90,116910.0,-25569.0,45092.0,1.71,-17.95,38.57,13.70,163.0,-0.57,0.88,37.99,OX40N,NTT,BANKS
46,KANSAINER,340.0,-67.97,58.0,H-SC,3.24,220644.0,-49023.0,85345.0,0.94,-18.18,38.68,13.47,138.0,-0.57,1.66,11.60,XY24,NTT,PAINTS
39,INDIGOPNTS,1408.0,113.17,48.0,M-SC,3.56,139946.0,-34613.0,34651.0,-1.47,-19.83,24.76,0.02,221.0,-1.00,1.05,21.73,OX40N,NTT,PAINTS
18,CERA,9475.0,-19.81,46.0,H-SC,2.53,114462.0,-29341.0,56086.0,0.28,-20.40,49.00,18.60,149.0,-0.52,0.86,26.31,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.69,38.0,H-LC,14.29,284124.0,-61832.0,185675.0,-0.84,-17.87,65.35,35.80,1.0,-0.33,2.14,0.65,X40,BTT,IT
41,INFY,2275.00,-21.39,38.0,H-LC,10.88,305783.0,-7105.0,178791.0,-0.62,-2.27,58.47,54.87,3.0,-0.04,2.30,4.23,X40,BTT,IT
43,ITC,452.00,-38.34,48.0,H-LC,3.13,199512.0,-626.0,21507.0,0.16,-0.31,10.78,10.44,4.0,-0.03,1.50,5.23,X40,NTT,FMCG
85,VBL,671.64,-16.13,38.0,H-LC,6.90,302029.0,-13813.0,126490.0,0.80,-4.37,41.88,35.67,5.0,-0.11,2.27,8.88,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,46.0,H-LC,12.91,247075.0,-14544.0,133766.0,0.59,-5.56,54.14,45.57,7.0,-0.11,1.86,5.13,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,4.14,82.0,H-MC,22.07,192920.0,42007.0,15299.0,2.78,27.83,7.93,37.97,93.0,2.75,1.45,60.18,AR,ATH,AUTO
25,DIXON,18940.15,-28.18,72.0,H-MC,7.99,143720.0,28784.0,7804.0,0.62,25.04,5.43,31.83,81.0,3.69,1.08,44.91,X40N,ATH,IT
26,DMART,5201.00,-7.03,63.0,H-LC,16.06,232172.0,47691.0,22683.0,-1.15,25.85,9.77,38.14,38.0,2.10,1.75,39.20,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,45.15,65.0,M-MC,7.54,228923.0,3961.0,163703.0,2.85,1.76,71.51,74.53,192.0,0.02,1.72,32.91,XY24,BTT,STEEL
14,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.31,188600.0,23930.0,44038.0,-2.96,14.53,23.35,41.28,89.0,0.54,1.42,23.67,X40N,ATH,AC
33,HEROMOTOCO,5949.07,4.14,82.0,H-MC,22.07,192920.0,42007.0,15299.0,2.78,27.83,7.93,37.97,93.0,2.75,1.45,60.18,AR,ATH,AUTO
86,VOLTAS,1918.49,1.30,60.0,H-MC,3.69,212490.0,20748.0,75285.0,-1.13,10.82,35.43,50.08,99.0,0.28,1.60,18.35,XY25,ATH,AC
31,HAVELLS,2069.16,-0.09,62.0,H-MC,1.89,247759.0,28.0,75021.0,0.36,0.01,30.28,30.30,92.0,0.00,1.87,13.30,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,4.14,82.0,H-MC,22.07,192920.0,42007.0,15299.0,2.78,27.83,7.93,37.97,93.0,2.75,1.45,60.18,AR,ATH,AUTO
51,NATIONALUM,247.44,-42.83,76.0,H-MC,1.31,113945.0,13441.0,17194.0,1.34,13.37,15.09,30.49,79.0,0.78,0.86,50.11,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.15,64.0,M-SC,3.64,102922.0,11424.0,67332.0,1.13,12.48,65.42,86.07,223.0,0.17,0.77,49.46,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.09,50.0,H-SC,12.11,136740.0,13404.0,120359.0,0.97,10.87,88.02,108.45,119.0,0.11,1.03,35.65,AR,ATH,MISC
88,WIPRO,420.00,-14.25,40.0,M-LC,6.88,150710.0,-235.0,109687.0,-0.24,-0.16,72.78,72.51,53.0,-0.00,1.13,5.86,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-172.73,67.0,M-SC,5.36,83442.0,-18768.0,112154.0,0.93,-18.36,134.41,91.37,208.0,-0.17,0.63,35.45,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,58.0,L-SC,38.12,86688.0,-26861.0,66914.0,1.32,-23.66,77.19,35.27,268.0,-0.40,0.65,112.17,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,45.0,H-SC,15.38,89010.0,-11953.0,106483.0,-0.17,-11.84,119.63,93.63,148.0,-0.11,0.67,32.58,SR,ATH,CHEMICALS
50,MASFIN,398.61,-17.34,54.0,H-SC,12.89,93855.0,-4125.0,25726.0,0.68,-4.21,27.41,22.05,152.0,-0.16,0.71,36.43,XR,ATH,FINANCE
84,VALIANTORG,838.00,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-41.65,58.0,L-SC,38.12,86688.0,-26861.0,66914.0,1.32,-23.66,77.19,35.27,268.0,-0.40,0.65,112.17,XR,NTT,HOTELS
7,ATULAUTO,844.00,3927.03,76.0,M-SC,8.64,125576.0,-21792.0,73613.0,-1.23,-14.79,58.62,35.16,236.0,-0.30,0.95,28.26,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,4.14,82.0,H-MC,22.07,192920.0,42007.0,15299.0,2.78,27.83,7.93,37.97,93.0,2.75,1.45,60.18,AR,ATH,AUTO
62,SAMMAANCAP,326.00,-172.73,67.0,M-SC,5.36,83442.0,-18768.0,112154.0,0.93,-18.36,134.41,91.37,208.0,-0.17,0.63,35.45,XY25,NTT,FINANCE
66,SHALBY,327.00,1097.77,79.0,M-SC,3.24,162862.0,-18281.0,63744.0,7.09,-10.09,39.14,25.10,235.0,-0.29,1.23,30.52,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.66
1,25,43.55
2,50,73.82


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.20
LC    35.93
MC    25.87
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.06
X40      15.51
X40N     14.19
XY25     10.09
AR        9.54
XR        9.09
OX40N     7.45
MH        1.79
X200      1.74
X5K       1.44
SR        1.10
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.66
H-MC    22.82
H-SC    22.70
M-SC    13.91
M-LC     7.17
M-MC     2.73
L-SC     1.59
L-LC     1.10
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.81,-0.55,35.07
IT,13.31,-16.74,74.62
FINANCE,7.46,-21.64,66.49
BANKS,7.39,-16.15,65.47
PAINTS,6.06,-14.65,40.32
MISC,5.88,-17.86,85.41
ELECTRICAL,5.70,-8.82,68.80
AUTO,4.89,-4.06,49.33
AC,3.75,2.78,30.14


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2739477.0,22
AR,1211606.0,10
XR,1084844.0,13
X40,906946.0,10
X40N,547452.0,11
OX40N,467659.0,10
XY25,411349.0,7
SR,197278.0,2
X5K,126734.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2758969.0,27
H-LC,1445766.0,16
M-SC,1397221.0,17
H-MC,1174302.0,17
M-LC,425892.0,5
M-MC,314756.0,2
L-SC,249981.0,3
L-MC,59873.0,1
L-LC,38771.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,924821.0,6
M-SC,XY24,814086.0,7
H-SC,AR,660556.0,5
H-LC,X40,617590.0,5
H-SC,XR,517970.0,6
H-MC,XY24,457618.0,4
H-LC,AR,377225.0,2
M-MC,XY24,314756.0,2
H-LC,X40N,238074.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 46.0 seconds
